In [14]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, mean_squared_error
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline

X_train = pd.read_csv('../data/X_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
Y_train = pd.read_csv('../data/Y_train.csv')
Y_test = pd.read_csv('../data/Y_test.csv')

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((2344, 136), (586, 136), (2344, 1), (586, 1))

# Linear regression

In [15]:
lr = LinearRegression()
lr.fit(X_train, Y_train)
test_pre = lr.predict(X_test)
train_pre = lr.predict(X_train)

print("Train Error: {:.3f}".format(mean_squared_error(y_true=Y_train,
                                                      y_pred=train_pre)))

print("Test Error : {:.3f}".format(mean_squared_error(y_true=Y_test,
                                                      y_pred=test_pre)))

Train Error: 726366079.635
Test Error : 1361050001.942
